In [9]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as Soup
import pandas as pd
from datetime import datetime

In [10]:
# 1) genome_scores() 1회 -> links() 1회 -> tags() 1회

# 2) Preprocessing() 1회 -> get_new_movies(Total) 반복 -> actor() 반복
#                                                      director() 반복

# 3) review() 1회

# 4) ratings() 1회

In [11]:
def genome_scores() :
    Genome_score = pd.read_csv('../../airflow_data/before_csv/genome-scores.csv')
    Genome_score = Genome_score.rename(columns = {'movieId' : 'movie_Id', 'tagId' : 'tag_Id'})
    
    links = pd.read_csv('../../airflow_data/before_csv/links.csv')
    links.drop(['tmdbId'], axis=1, inplace=True)
    links.rename(columns = {'movieId':'movie_Id'}, inplace = True)
    
    df = pd.merge(Genome_score, links, on = 'movie_Id')
    df = df[['imdbId', 'tag_Id', 'relevance']]
    df = df.reset_index()
    df = df.rename(columns = {'index' : 'scores_Id', 'imdbId' : 'imdb_movie_Id'})
    df = df.set_index('scores_Id')
    
    return df.to_csv('../../airflow_data/after_csv/genome_score.csv')




In [12]:
def links() :
    links = pd.read_csv('./병합 전 csv/links.csv')
    links.drop(['tmdbId'], axis=1, inplace=True)
    links.rename(columns = {'movieId':'movie_Id'}, inplace = True)
    links = links.rename(columns = {'imdbId' : 'imdb_movie_Id'})
    links = links.set_index('movie_Id')
    
    return links.to_csv('./병합 후 csv/links.csv')

In [13]:
## 전처리 함수 1회만 실행

def Preprocessing() :
    global Total
    # movies.csv
    # links.csv 은 최초 원본파일
    
    movie = pd.read_csv('./병합 전 csv/movies.csv')
    links = pd.read_csv('./병합 전 csv/links.csv') 
    imdb_movie = pd.merge(movie, links, on = 'movieId')
    imdb_movie = imdb_movie[['imdbId', 'title', 'genres']]
    imdb_movie = imdb_movie.rename(columns = {'imdbId':'imdb_movie_Id'})
    
    movie2000 = pd.read_csv('./2000-01~Current Top 1000 Movies.csv')
    movie2000 = movie2000[['Movie_No','Name','Genre']]
    movie2000 = movie2000.rename(columns = {'Movie_No':'imdb_movie_Id', 'Name':'title', 'Genre':'genres'})

    movie2020 = pd.read_csv('./2020_Movie_list.csv')
    movie2020 = movie2020[['Movie_No','Name','Genre', 'Director', 'Actor_list']]
    movie2020 = movie2020.rename(columns = {'Movie_No':'imdb_movie_Id', 'Name':'title', 'Genre':'genres', 'Director':'Director_name', 'Actor_list':'Actor_name'})

    Total = pd.concat([imdb_movie, movie2000, movie2020], axis = 0)
    return Total

In [14]:
def get_new_movies(Total) :
    Name = []
    MovieId = []
    Year = []
    Rating = []
    Meta_score = []
    Votes = []
    Genre = []
    Director = []
    Stars = []
    
    Date = datetime.today().strftime("%Y-%m")    # YYYYmmddHHMMSS 형태의 시간 출력

    
    url = get('https://www.imdb.com/movies-coming-soon/' + Date + '/?ref_=cs_dt_pv')
    request = url.text
    soup_data = Soup(request, 'html.parser')

    movies = soup_data.findAll('div', {'class' : 'list_item odd'})
    movies1 = soup_data.findAll('div', {'class' : 'list_item even'})
    for i in movies + movies1:
        # 영화 제목
        Name.append(i.h4.a.text[:-7])
        # 영화 년도
        Year.append(i.h4.a.text[-5:-1])
        # 영화 ID
        MovieId.append(i.h4.a["href"][-8:-1])
        # 영화 장르
        tmp = i.select('p> span')
        tmp = list(filter(lambda x: str(x)[:6] == '<span>', tmp))
        tmp = list(map(lambda x: str(x).replace('<span>','').replace('</span>',''), tmp))
        Genre.append(tmp)
        # 영화 감독
        Director.append(i.find('div', {'class' : 'txt-block'}).span.text.strip())
        # 영화 배우

        Stars_ = list(map(lambda x: str(x).replace('</a>',''), i.select('div.txt-block> a')))
        result = []
        for e in Stars_:
            sign = False
            a_result = ''

            for c in e :
                if sign:
                    a_result += c

                elif c == '>':
                    sign = True

            result.append(a_result)

        Stars.append(result)


        # Meta Score
        try :
            Meta_score.append(i.tr.find('div', {'class':'rating_txt'}).span.text.strip())
        except Exception as err:
            Meta_score.append(0)
    
    data = list(zip(MovieId, Name, Year, Genre, Meta_score, Director,Stars))
    df = pd.DataFrame(data, columns = ['imdb_movie_Id', 'title', 'Year', 'genres','Meta_score', 'Director_name','Actor_name'])
    df = df[['imdb_movie_Id', 'title', 'genres', 'Director_name', 'Actor_name']]
    Total = pd.concat([Total,df], axis = 0)
    Total = Total.drop_duplicates(['imdb_movie_Id'])
    Total = Total.set_index('imdb_movie_Id')
    
    return Total.to_csv('./병합 후 csv/Total.csv')

In [15]:
def tags() :
    tags = pd.read_csv('./병합 전 csv/tags.csv')
    links = pd.read_csv('./병합 후 csv/links.csv', index_col = 'movie_Id')
    
    tags.rename(columns = {'movieId' : 'movie_Id', 'userId':'user_Id'}, inplace = True)
    tags = pd.merge(tags, links, on = 'movie_Id')
    tags = tags.sort_values(by='user_Id' ,ascending=True)
    tags = tags.reset_index()
    tags = tags.reset_index()
    tags = tags.rename(columns = {'level_0' : 'tags_No'})
    tags = tags.set_index('tags_No')
    tags = tags[['imdb_movie_Id', 'user_Id', 'tag']]
    
    return tags.to_csv('./병합 후 csv/tags.csv')

In [16]:
def actor() :
    Total = pd.read_csv('./병합 후 csv/total.csv', index_col='imdb_movie_Id')
    b = list(Total['Actor_name'].dropna())
    
    actor = []
    for i in range(len(b)) :
        for j in range(len(b[i][1:-1].split(','))) :
            if j == 0 :
                actor.append(b[i][1:-1].split(',')[j][1:-1])
            else :
                actor.append(b[i][1:-1].split(',')[j][2:-1])
                
    df_actor = pd.DataFrame(actor)
    df_actor = df_actor.drop_duplicates()
    df_actor = df_actor.reset_index()
    df_actor = df_actor.reset_index()
    df_actor = df_actor.rename(columns = {0 : 'Actor_name', 'level_0' : 'Actor_Id'})
    df_actor = df_actor.drop('index', axis = 1)
    df_actor = df_actor.set_index('Actor_Id')
    return df_actor.to_csv('./병합 후 csv/Actor.csv')

In [17]:
def director() :
    Total = pd.read_csv('./병합 후 csv/total.csv', index_col='imdb_movie_Id')
    Director = Total[['Director_name']]
    Director = Director.dropna()
    Director = Director.reset_index()
    Director = Director.drop(['imdb_movie_Id'], axis = 1)
    Director = Director.reset_index()
    Director = Director.rename(columns = {'index' : 'Director_Id'})
    Director = Director.set_index('Director_Id')
    
    return Director.to_csv('./병합 후 csv/Director.csv')

In [18]:
def review() :
    review = pd.read_csv('./병합 전 csv/Review_Crawl.csv')
    review = review.rename(columns = {'Movie_number' : 'imdb_movie_Id', 'Review_text' : 'review', 'Unnamed: 0' : 'Review_No'})
    review = review.set_index('Review_No')
    
    return review.to_excel('./병합 후 csv/review.xlsx')

In [19]:
def ratings() :
    movie = pd.read_csv('./병합 전 csv/20-01~21-09 Top 1000 평점데이터 수정전.csv', index_col=0)
    ratings = pd.read_csv('./병합 전 csv/ratings.csv')
    links = pd.read_csv('./병합 전 csv/links.csv')
    
    movie.drop('User_ID', axis=1, inplace = True)
    for i in range(len(movie.User_rating)) :
        if movie.User_rating[i] not in ['0','1','2','3','4','5','6','7','8','9','10'] :
            movie.loc[i,'User_rating'] = '0'

    movie = movie.rename(columns = {'User_rating_movie':'imdb_movie_Id', 'User_rating':'rating', 'UserID':'user_Id'})
    
    ratings.drop(['timestamp'], axis=1,inplace=True)
    ratings['rating'] = ratings['rating'] * 2
    ratings = pd.merge(ratings, links, on = 'movieId')
    ratings.drop(['movieId'], axis =1, inplace=True)
    ratings.rename(columns = {'index' : 'ratings_Id', 'userId':'user_Id', 'imdbId':'imdb_movie_Id'}, inplace = True)
    
    links.drop(['tmdbId'], axis=1, inplace=True)
    ratings_ = pd.concat([ratings, movie], axis = 0)
    ratings_ = ratings_.reset_index()
    ratings_.drop(['index'], axis=1, inplace=True)
    ratings_ = ratings_.reset_index()
    ratings_.rename(columns = {'index' : 'ratings_Id'}, inplace = True)
    ratings_.set_index('ratings_Id', inplace=True)
    ratings_.drop(['tmdbId'], axis=1, inplace=True)
    return ratings_.to_csv('./병합 후 csv/ratings_.csv')